## Langchain

This notebook is an initial experimentation with Langchain.

### Components of LangChain

1. **Prompts**: You can define prompt templates, so you take the user input and use the templates to create the final prompt for the model 
2. **Chains**: Chains are sequences of LLM calls.  For example: chain together prompt templates and LLMs. There are many-to-many combinations of how you can chain components together. 
3. **Memory**: By default, Chains and Agents are stateless, meaning that they treat each incoming query independently (as are the underlying LLMs and chat models). Langchain provides memory interfaces to allow you to store history of conversations. 
4. **Indexes**: Utility functions to allow you to provide your own text data. For example, it provides data loaders to allow you load in your own PDF files, emails etc. It also provides vector store interfaces to store the data in a vector database - Making it easily retrievable for the model/chain.  
5. **Agents/Tools**: You can set up agents powered by large language models that can use tools such as google search, wikipedia or calculators.  

<img src="./img/lc.png" style="height:500px; display: block; margin-right: auto; margin-left: auto;">



---

- [Docs](https://langchain.readthedocs.io/en/latest/index.html)
- [Github Repo](https://github.com/hwchase17/langchain)
- [LangChain explained - The hottest new Python framework](https://www.youtube.com/watch?v=RoR4XJw8wIc)


In [1]:
#|exporti

import os

from nbdev.export import nb_export
import streamlit as st
from streamlit_jupyter import StreamlitPatcher, tqdm

StreamlitPatcher().jupyter()  

from langchain.chains import LLMChain, SimpleSequentialChain 
from langchain.llms import OpenAI 
from langchain.prompts import PromptTemplate 

### Set up Open AI

In [2]:
#|exporti

# If an API key has been provided, create an OpenAI language model instance
API = os.getenv('OPENAI_KEY')

if API:
    llm = OpenAI(temperature=0.7, openai_api_key=os.getenv('OPENAI_KEY'))
else:
    # If an API key hasn't been provided, display a warning message
    st.warning("OPENAI_KEY Not Found")


2023-04-21 13:21:03.981 
  command:

    streamlit run /Users/award40/opt/anaconda3/envs/dev/lib/python3.9/site-packages/ipykernel_launcher.py [ARGUMENTS]


### Langchain Components

### Web Application

In [3]:
#|exporti

# Create simple web app using Streamlit

# Set the title of the Streamlit app
st.title("⛓ LangChain Example")

# Add a link to the Github repository that inspired this app
st.markdown("Enter a question below to ask ChatGPT")

# ✅ What's TRUE  : Using LangChain `SimpleSequentialChain`

Inspired from [fact-checker](https://github.com/jagilley/fact-checker) by Jagiley

In [4]:
#|exporti

# Add a text input box for the user's question
user_question = st.text_input(
    "Enter Your Question : ",
    placeholder = "How was your day?",
)

Textarea(value='', description='Enter Your Question : ', placeholder='Type something')

In [ ]:
# Chain 1: Generating a rephrased version of the user's question
template = """{question}\n\n"""
prompt_template = PromptTemplate(input_variables=["question"], template=template)
question_chain = LLMChain(llm=llm, prompt=prompt_template)

# Chain 2: Generating assumptions made in the statement
template = """Here is a statement:
    {statement}
    Make a bullet point list of the assumptions you made when producing the above statement.\n\n"""
prompt_template = PromptTemplate(input_variables=["statement"], template=template)
assumptions_chain = LLMChain(llm=llm, prompt=prompt_template)
assumptions_chain_seq = SimpleSequentialChain(
    chains=[question_chain, assumptions_chain], verbose=True
)

# Chain 3: Fact checking the assumptions
template = """Here is a bullet point list of assertions:
{assertions}
For each assertion, determine whether it is true or false. If it is false, explain why.\n\n"""
prompt_template = PromptTemplate(input_variables=["assertions"], template=template)
fact_checker_chain = LLMChain(llm=llm, prompt=prompt_template)
fact_checker_chain_seq = SimpleSequentialChain(
    chains=[question_chain, assumptions_chain, fact_checker_chain], verbose=True
)

# Final Chain: Generating the final answer to the user's question based on the facts and assumptions
template = """In light of the above facts, how would you answer the question '{}'""".format(
    user_question
)
template = """{facts}\n""" + template
prompt_template = PromptTemplate(input_variables=["facts"], template=template)
answer_chain = LLMChain(llm=llm, prompt=prompt_template)
overall_chain = SimpleSequentialChain(
    chains=[question_chain, assumptions_chain, fact_checker_chain, answer_chain],
    verbose=True,
)

In [5]:
#|exporti

# Generating the final answer to the user's question using all the chains
if st.button("Send", type="primary"):

    # Running all the chains on the user's question and displaying the final answer
    st.success(overall_chain.run(user_question))

### Export Streamlit components as py file

In [6]:
nb_export('example.ipynb', lib_path='./', name='example')